Se requiere obtener información el "total de presupuesto" y "total ingresos" de las "películas" donde el "año de la fehca de lanzamiento" debe ser mayor o igual a 2015 tambien debe estar agrupado por el "año de la fecha de lanzamiento" y el "pais" donde se realizó la película.

También se requiere realizar un ranking ordenado de manera "descendente" por "el total de presupuesto" y "total de ingresos" participando por el "Año de la fecha de lanzamiento"

### Leer todos los datos que son requeridos

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
movies_df = spark.read.format("delta").load(f"{silver_folder_path}/movies") \
                       .filter(f"file_date = '{v_file_date}'")
#movies_df = spark.read.parquet(f"{silver_folder_path}/movies") \
#    .filter(f"file_date = '{v_file_date}'")

In [0]:
countries_df = spark.read.format("delta").load(f"{silver_folder_path}/countries")
#countries_df = spark.read.parquet(f"{silver_folder_path}/countries")

In [0]:
productions_countries_df = spark.read.format("delta").load(f"{silver_folder_path}/productions_countries") \
                        .filter(f"file_date = '{v_file_date}'")
#productions_countries_df = spark.read.parquet(f"{silver_folder_path}/productions_countries") \
#    .filter(f"file_date = '{v_file_date}'")

### Join "countries" y "productions_countries"

In [0]:
countries_prod_coun_df = countries_df.join(productions_countries_df, countries_df.country_id == productions_countries_df.country_id,"inner") \
  .select("country_name", "movie_id")

### Join "movies_df" y "countries_prod_coun_df"

- Filtrar las películas dónde su fecha de lanzamiento sea mayor o igual a 2015

In [0]:
movies_filter_df = movies_df.filter("year_release_date >= 2015")

In [0]:
result_movies_countries_df = movies_filter_df.join(countries_prod_coun_df, movies_filter_df.movie_id == countries_prod_coun_df.movie_id,"inner")

In [0]:
results_df = result_movies_countries_df.select("year_release_date", "country_name", "budget", "revenue")

In [0]:
from pyspark.sql.functions import sum
results_group_by_df = results_df \
.groupBy("year_release_date", "country_name") \
.agg(
    sum("budget").alias("total_budget"), 
    sum("revenue").alias("total_revenue")
)

In [0]:
from pyspark.sql.functions import lit

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank, desc
result_dense_rank_df = Window.partitionBy("year_release_date").orderBy(
                                                                desc("total_revenue"),
                                                                desc("total_revenue"))
final_df = results_group_by_df.withColumn("dense_rank", dense_rank().over(result_dense_rank_df)) \
            .withColumn("created_date", lit(v_file_date))

### Escribir datos en el DataLake en formato "Parquet"

In [0]:
#overwrite_partition(final_df, "movie_gold", "results_group_movie_country", "created_date")

In [0]:
merge_condition = 'tgt.year_release_date = src.year_release_date AND tgt.country_name = src.country_name AND tgt.created_date = src.created_date'
merge_delta_lake(final_df, "movie_gold", "results_group_movie_country", gold_folder_path, merge_condition, "created_date")
#final_df.write.mode("append").partitionBy("created_date").format("parquet").saveAsTable("movie_gold.results_group_movie_country")
#final_df.write.mode("overwrite").parquet(f"{gold_folder_path}/results_group_movie_country")

In [0]:
%sql
SELECT * FROM movie_gold.results_group_movie_country;

In [0]:
#display(spark.read.parquet(f"{gold_folder_path}/results_group_movie_country"))